<a href="https://colab.research.google.com/github/ihashirjadoon/llama3-hackathon/blob/main/Notebooks/Llama_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.4.0 which is incompatible.


#Get API Keys and add them before running


In [80]:
import os
from together import Together
from google.colab import userdata
import re
import json
# Import necessary libraries
import google.generativeai as genai


#Add API Keys before running
genaiAPI = ""
client = Together(api_key='')

#Inputs from Frontend

1. Book Type : book_type (Novel, Short story or anything)
2. Title in Mind : book_title
3. Input Data: entire_input_data (I am using some paragraphs from a book for trial run)

In [81]:
# Get user input for the type of book and title
book_type = input("Enter the type of book (e.g., novel, short story, etc.): ")
book_title= input("Enter the title in your mind: ")

Enter the type of book (e.g., novel, short story, etc.): novel
Enter the title in your mind: I Feel Bad About My Neck


In [82]:

# In a real scenario, this would be read from a file or other input source
entire_input_data = """
I feel bad about my neck. Truly I do. If you saw my neck, you might feel bad about it too, but you’d probably be too polite to let on. If I said something to you on the subject—something like “I absolutely cannot stand my neck”—you’d undoubtedly respond by saying something nice, like “I don’t know what you’re talking about.” You’d be lying, of course, but I forgive you. I tell lies like that all the time—mostly to friends who tell me they’re upset because they have little pouches under their eyes, or jowls, or wrinkles, or flab around the middle, and do I think they should have an eye job, or a face-lift, or Botox, or liposuction. My experience is that “I don’t know what you’re talking about” is code for “I see what you mean, but if you think you’re going to trap me into engaging on this subject, you’re crazy.” It’s dangerous to engage on such subjects, and we all know it. Because if I said, “Yes, I see exactly what you mean,” my friend might go out and have her eyes done, for instance, and it might not work, and she might end up being one of those people you read about in tabloids who ends up in court suing their plastic surgeons because they can never close their eyes again. Furthermore, and this is the point: It would be All My Fault. I am particularly sensitive to the All My Fault aspect of things, since I have never forgiven one of my friends for telling me not to buy a perfectly good apartment on East Seventy-fifth Street in 1976.
Sometimes I go out to lunch with my girlfriends—I got that far into the sentence and caught myself. I suppose I mean my women friends. We are no longer girls and have not been girls for forty years. Anyway, sometimes we go out to lunch and I look around the table and realize we’re all wearing turtleneck sweaters. Sometimes, instead, we’re all wearing scarves, like Katharine Hepburn in On Golden Pond. Sometimes we’re all wearing mandarin collars and look like a white ladies’ version of the Joy Luck Club. It’s sort of funny and it’s sort of sad, because we’re not neurotic about age —none of us lies about how old she is, for instance, and none of us dresses in a way that’s inappropriate for our years. We all look good for our age. Except for our necks.
Oh, the necks. There are chicken necks. There are turkey gobbler necks. There are elephant necks. There are necks with wattles and necks with creases that are on the verge of becoming wattles. There are scrawny necks and fat necks, loose necks, crepey necks, banded necks, wrinkled necks, stringy necks, saggy necks, flabby necks, mottled necks. There are necks that are an amazing combination of all of the above. According to my dermatologist, the neck starts to go at forty-three, and that’s that. You can put makeup on your face and concealer under your eyes and dye on your hair, you can shoot collagen and Botox and Restylane into your wrinkles and creases, but short of surgery, there’s not a damn thing you can do about a neck. The neck is a dead giveaway. Our faces are lies and our necks are he truth. You have to cut open a redwood tree to see how old it is, but you wouldn’t have to if it had a neck.

"""

#Functions

##summarize_paragraphs:

Model: Gemini Flash

Purpose: Summarizing all the previous paragraphs for making the model efficient


##get_last_paragraph_and_summary

For finding the last paragraph, paragraph number, and the previous paragraphs for the summarize function


##Prompt

The prompt being used for our Llama Model


##jsonResponse:

Extracting the json content from the Llama3

In [90]:
def summarize_paragraphs(paragraphs, api_key):
  """
  Summarizes a text paragraph using Gemini Flash (requires API key).

  Args:
      text: The text paragraph to summarize.
      api_key: Your Google Generative AI API key.

  Returns:
      A string containing the summarized text.
  """

  # Configure GenAI with your API key
  genai.configure(api_key=api_key)

  # Load the Gemini Flash model
  model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

  # Define the prompt for summarization
  prompt = f"Summarize the following text: {paragraphs}"

  # Generate the summary using Gemini Flash
  response = model.generate_content(prompt)

  # Extract the summarized text from the response
  summary = response.text.split("\n")[0].strip()  # Assuming first line is summary
  return summary

# Function to get the last paragraph (node) and its number from the entire input data
def get_last_paragraph_and_summary(data):
    paragraphs = data.strip().split('\n')
    last_paragraph = paragraphs[-1] if paragraphs else ""
    previous_paragraphs = paragraphs[:-1] if len(paragraphs) > 1 else []
    summary = summarize_paragraphs(previous_paragraphs, genaiAPI)
    paragraph_number = len(paragraphs)
    return summary, last_paragraph, paragraph_number


# Extract the summary of previous paragraphs, the last paragraph (node), and its number
summary, last_paragraph, paragraph_number = get_last_paragraph_and_summary(entire_input_data)

# Define the input message for analysis, incorporating user inputs, the identified paragraph number, and the summary
prompt = f"""
I am writing a {book_type} and This is paragraph {paragraph_number}. The title in my mind is {book_title}.
Here is a summary of the previous paragraphs: {summary}
Analyse the last paragraph and give me ways to improve: {last_paragraph}.
After analysing, give a json output for :
'Overall interest' [A floating value out of 10],
Vivid Rating [A floating value out of 10],
Build up: [Strong, Weak, Average],
Irregularities: [if any],
References: [Some reference book titles and authors],
Summary : (summary of the paragraph),
Suggestions: ,
Strengths,
Weaknesses.
"""


# Request analysis and suggestions from the model
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
    messages=[
        {"role": "user", "content": prompt}
    ],
    stream=True,
)

# Initialize an empty string to store the response
response=""
# Print the response from the model
for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    print(chunk.choices[0].delta.content or "", end="", flush=True)

Here's the analysis of the paragraph:

**Overall interest: 8.5/10**
The paragraph is engaging and humorous, with a relatable topic and vivid descriptions of different types of necks. The use of metaphors (e.g., "chicken necks", "turkey gobbler necks") adds to the entertainment value.

**Vivid Rating: 9/10**
The descriptions of necks are creative and evocative, making it easy for the reader to imagine the different types of necks. The use of alliteration (e.g., "scrawny necks and fat necks") adds to the vividness.

**Build up: Strong**
The paragraph builds up well, starting with a general statement about necks and then providing a list of different types of necks. The transition to the dermatologist's statement and the conclusion about the neck being a "dead giveaway" is smooth.

**Irregularities: None**
The paragraph is well-structured and easy to follow.

**References:**
* "The Beauty Myth" by Naomi Wolf (a book that explores the societal pressure on women to conform to beauty standar

In [115]:
def jsonResponse(response, api_key):

  # Configure GenAI with your API key
  genai.configure(api_key=api_key)

  # Load the Gemini Flash model
  model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

  # Define the prompt for summarization
  prompt = f"Make this {response} as a json with keys 'Overall interest','Vivid Rating','Build up','Irregularities','References' (Dont use any special characters here),'Summary','Suggestions','Strengths','Weaknesses'"

  # Generate the summary using Gemini Flash
  response = model.generate_content(prompt)

  jsonTry = response.text
  return jsonTry

# Function to get the last paragraph (node) and its number from the entire input data
jsonRaw=jsonResponse(response, genaiAPI)
jsonRaw=jsonRaw.replace("\n", "").replace("  ", "").replace("json", "").replace("`", "")
jsonRaw

'{"Overall interest": "8.5/10","Vivid Rating": "9/10","Build up": "Strong","Irregularities": "None","References": "The Beauty Myth by Naomi Wolf (a book that explores the societal pressure on women to conform to beauty standards)\\nThe Feminine Mystique by Betty Friedan (a classic feminist book that touches on the theme of women\'s bodies and aging)","Summary": "The paragraph humorously describes the various types of necks and how they are a giveaway of age. The author notes that while makeup and other beauty treatments can hide facial imperfections, the neck is a harder area to conceal.","Suggestions": "Consider adding more sensory details to make the descriptions even more vivid. For example, you could describe the texture or color of the different types of necks.\\nYou may want to vary your sentence structure to create a more dynamic rhythm. The paragraph is heavy on simple sentences; consider mixing in some compound or complex sentences to create more interest.\\nWhile the paragrap

In [116]:
jsonData=json.loads(jsonRaw)
jsonData

{'Overall interest': '8.5/10',
 'Vivid Rating': '9/10',
 'Build up': 'Strong',
 'Irregularities': 'None',
 'References': "The Beauty Myth by Naomi Wolf (a book that explores the societal pressure on women to conform to beauty standards)\nThe Feminine Mystique by Betty Friedan (a classic feminist book that touches on the theme of women's bodies and aging)",
 'Summary': 'The paragraph humorously describes the various types of necks and how they are a giveaway of age. The author notes that while makeup and other beauty treatments can hide facial imperfections, the neck is a harder area to conceal.',
 'Suggestions': 'Consider adding more sensory details to make the descriptions even more vivid. For example, you could describe the texture or color of the different types of necks.\nYou may want to vary your sentence structure to create a more dynamic rhythm. The paragraph is heavy on simple sentences; consider mixing in some compound or complex sentences to create more interest.\nWhile the p